In [1]:
import bs4
from urllib.request import urlopen
from bs4 import BeautifulSoup as soup
import pandas as pd

In [2]:
myUrl="https://www.moneyworks4me.com/best-index/nse-stocks/top-nifty50-companies-list/"
client=urlopen(myUrl)
page_html=client.read()
client.close()

In [3]:
raw_data=soup(page_html,'html.parser')

In [4]:
table_container_one=raw_data.find_all("div",{"class":"responsive-holder","id":"stock-data"})
table_container_two=table_container_one[0].find_all('tbody')
table_container_three=table_container_two[0].find_all('tr',{'class':'countRow'})

In [5]:
columns=['CompanyName','RightStock?','CMP','PriceChange%','MarketCap','52WHigh','52WLow','ROE','P/E','P/BV','EV/EBITDA']
df=pd.DataFrame(columns=columns,index=[x for x in range(len(table_container_three))])
def process(text):
    if ',' in text:
        text=text.split(',')
        text=float(text[0]+text[1])
    elif text=='-':
        return 0
    else:
        text=float(text)
    return text
for i,row in enumerate(table_container_three):
    name=row.find('a').text
    rightStock=row.find_all('td')[1].find_all('span')[-1].text
    cmp=process(row.find_all('td')[2].text)
    priceChange=process(row.find_all('td')[3].text[:-1])
    marketCap=process(row.find_all('td')[4].text)
    wHigh=process(row.find_all('td')[5].text)
    wLow=process(row.find_all('td')[6].text)
    roe=process(row.find_all('td')[7].text)
    p_e=process(row.find_all('td')[8].text)
    p_bv=process(row.find_all('td')[9].text)
    ev=process(row.find_all('td')[10].text)
    df.loc[i]=[name,rightStock,cmp,priceChange,marketCap,wHigh,wLow,roe,p_e,p_bv,ev]
#df['RightStock?'].replace(['Very Good','Somewhat Good','Not Good'],[2,1,0],inplace=True)
df.to_csv('NIFTY50.csv',index=False)